# GRU: Gated Recurrent Unit

## Understanding GRU Networks

**Based on the concepts from "Understanding LSTM Networks" by Christopher Olah**

### Introduction

In the previous notebook, we learned about LSTMs and how they solve the vanishing gradient problem through gating mechanisms. However, LSTMs are quite complex with 4 different gates.

**GRU (Gated Recurrent Unit)**, introduced by Cho et al. in 2014, is a simpler alternative that:
- Uses only **2 gates** instead of 4
- Merges the cell state and hidden state
- Has **fewer parameters** (faster training)
- Often achieves **similar performance** to LSTM

### Learning Objectives

1. Understand how GRU simplifies LSTM architecture
2. Learn the two gates: reset and update
3. Implement GRU from scratch
4. Compare GRU vs LSTM performance
5. Understand when to use GRU vs LSTM

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

---

## 1. From LSTM to GRU: Simplification

### 1.1 Recap: LSTM Architecture

LSTM has **4 components**:

```
┌────────────────────────────────────────────────────────────┐
│                    LSTM (4 gates)                          │
└────────────────────────────────────────────────────────────┘

1. Forget gate (f_t):  What to forget from cell state
2. Input gate (i_t):   What new info to add
3. Cell gate (C̃_t):    Candidate values
4. Output gate (o_t):  What to output

Plus: Separate cell state (C_t) and hidden state (h_t)
```

### 1.2 GRU Simplification

GRU simplifies this to **2 gates**:

```
┌────────────────────────────────────────────────────────────┐
│                    GRU (2 gates)                           │
└────────────────────────────────────────────────────────────┘

1. Reset gate (r_t):   How much past info to forget
2. Update gate (z_t):  How much to update with new info

Plus: Merges cell state and hidden state into one (h_t)
```

### Key Differences:

| Feature | LSTM | GRU |
|---------|------|-----|
| **Gates** | 4 (forget, input, cell, output) | 2 (reset, update) |
| **States** | 2 (cell C_t, hidden h_t) | 1 (hidden h_t) |
| **Parameters** | More | Fewer (~25% less) |
| **Training Speed** | Slower | Faster |
| **Performance** | Excellent | Similar to LSTM |
| **Use Case** | When you have lots of data | When you want faster training |

---

## 2. GRU Architecture: Step-by-Step

### 2.1 The Big Picture

```
┌────────────────────────────────────────────────────────────┐
│                    GRU CELL STRUCTURE                      │
└────────────────────────────────────────────────────────────┘

Inputs: x_t (current input), h_{t-1} (previous hidden)

                    ┌─────────────────┐
                    │  RESET GATE     │
                    │  r_t = σ(...)   │  ← How much past to use
                    └────────┬────────┘
                             │
    h_{t-1} ─────────────────×─────────┐
    (previous)               │          │
                             │          │
                    ┌────────▼────────┐ │
                    │  CANDIDATE      │ │
                    │  h̃_t = tanh(...) │ │  ← New candidate
                    └────────┬────────┘ │
                             │          │
                    ┌────────▼────────┐ │
                    │  UPDATE GATE    │ │
                    │  z_t = σ(...)   │ │  ← How much to update
                    └────────┬────────┘ │
                             │          │
                             ▼          ▼
                    h_t = (1-z_t)⊙h_{t-1} + z_t⊙h̃_t
                          (new hidden state)

Legend:
  σ = sigmoid (0 to 1) - acts as a gate
  × = element-wise multiplication
  ⊙ = element-wise multiplication
  tanh = hyperbolic tangent (-1 to 1)
```

### 2.2 Step-by-Step Walk Through

Let's understand each component with intuitive examples.

#### **Step 1: Reset Gate** (How much past to forget)

```
r_t = σ(W_r · [h_{t-1}, x_t] + b_r)
```

**What it does:**
- Decides how much of the previous hidden state to use
- Output: Values between 0 and 1
- **0** = "Completely ignore the past"
- **1** = "Completely use the past"

**Example:** Language modeling
- When starting a new sentence, reset gate → 0 (forget previous sentence)
- When continuing a sentence, reset gate → 1 (use previous context)

```
┌────────────────────────────────────────────────────────────┐
│              RESET GATE VISUALIZATION                      │
└────────────────────────────────────────────────────────────┘

Sentence 1: "The cat sat on the mat."
Sentence 2: "Dogs are loyal."
                    ↑
            Reset gate ≈ 0
         (New sentence, forget previous)

Within sentence: "The cat sat on the..."
                                    ↑
                        Reset gate ≈ 1
                    (Continue, use context)
```

#### **Step 2: Candidate Hidden State** (What new info to consider)

```
h̃_t = tanh(W_h · [r_t ⊙ h_{t-1}, x_t] + b_h)
```

**What it does:**
- Creates a candidate for the new hidden state
- Uses **reset gate** to control how much past info to include
- Output: Values between -1 and 1 (tanh)

**Key insight:**
- `r_t ⊙ h_{t-1}` means: "Use only the relevant parts of the past"
- If reset gate is 0, we ignore the past completely
- If reset gate is 1, we use all the past information

```
┌────────────────────────────────────────────────────────────┐
│           CANDIDATE STATE COMPUTATION                      │
└────────────────────────────────────────────────────────────┘

Case 1: Reset gate ≈ 0 (New context)
  r_t ⊙ h_{t-1} ≈ 0
  h̃_t ≈ tanh(W_h · [0, x_t])
  → Candidate based mostly on current input

Case 2: Reset gate ≈ 1 (Continue context)
  r_t ⊙ h_{t-1} ≈ h_{t-1}
  h̃_t ≈ tanh(W_h · [h_{t-1}, x_t])
  → Candidate uses both past and current
```

#### **Step 3: Update Gate** (How much to update)

```
z_t = σ(W_z · [h_{t-1}, x_t] + b_z)
```

**What it does:**
- Decides how much to update the hidden state
- Output: Values between 0 and 1
- **0** = "Keep old hidden state completely"
- **1** = "Use new candidate completely"

**Example:** Counting task
- When seeing a relevant item, update gate → 1 (update count)
- When seeing irrelevant item, update gate → 0 (keep old count)

```
┌────────────────────────────────────────────────────────────┐
│              UPDATE GATE VISUALIZATION                     │
└────────────────────────────────────────────────────────────┘

Task: Count number of 'a's in text

Input: "b c a d a e f"
        ↓ ↓ ↓ ↓ ↓ ↓ ↓
Update: 0 0 1 0 1 0 0
        │ │ │ │ │ │ │
Count:  0 0 1 1 2 2 2
            ↑   ↑
    Update when 'a' appears
```

#### **Step 4: Final Hidden State** (Combine old and new)

```
h_t = (1 - z_t) ⊙ h_{t-1} + z_t ⊙ h̃_t
      ↑                     ↑
   Keep old            Use new
```

**What it does:**
- Interpolates between old hidden state and new candidate
- Update gate controls the balance

**Key insight:**
- This is a **weighted average** between old and new
- `(1 - z_t)` and `z_t` always sum to 1
- Ensures smooth transitions

```
┌────────────────────────────────────────────────────────────┐
│           FINAL HIDDEN STATE COMPUTATION                   │
└────────────────────────────────────────────────────────────┘

Example with z_t = 0.3:

h_t = 0.7 × h_{t-1} + 0.3 × h̃_t
      ↑               ↑
   70% old        30% new

Extreme cases:
• z_t = 0: h_t = h_{t-1}  (no update, keep old)
• z_t = 1: h_t = h̃_t      (full update, use new)
• z_t = 0.5: h_t = 0.5×h_{t-1} + 0.5×h̃_t  (equal mix)
```

### 2.3 Complete GRU Equations

Putting it all together:

$$
\begin{align}
r_t &= \sigma(W_r \cdot [h_{t-1}, x_t] + b_r) && \text{(Reset gate)} \\
z_t &= \sigma(W_z \cdot [h_{t-1}, x_t] + b_z) && \text{(Update gate)} \\
\tilde{h}_t &= \tanh(W_h \cdot [r_t \odot h_{t-1}, x_t] + b_h) && \text{(Candidate)} \\
h_t &= (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t && \text{(Final hidden)}
\end{align}
$$

**Notation:**
- $\sigma$ = sigmoid function (0 to 1)
- $\tanh$ = hyperbolic tangent (-1 to 1)
- $\odot$ = element-wise multiplication
- $[a, b]$ = concatenation of vectors a and b

---

## 3. Why GRU Works: Gradient Flow

### 3.1 Comparison with Vanilla RNN

```
┌────────────────────────────────────────────────────────────┐
│         GRADIENT FLOW: RNN vs GRU                          │
└────────────────────────────────────────────────────────────┘

Vanilla RNN:
  h_t = tanh(W_hh · h_{t-1} + W_xh · x_t)
  
  Gradient: ∂L/∂h_0 = ∂L/∂h_T · ∏(W_hh · tanh')
                                  ↑
                          Repeated multiplication
                          → Vanishing gradient!

GRU:
  h_t = (1 - z_t) ⊙ h_{t-1} + z_t ⊙ h̃_t
  
  Gradient: ∂L/∂h_0 = ∂L/∂h_T · ∏(1 - z_t)
                                  ↑
                          Controlled by gates
                          → Can preserve gradient!

Key Insight:
• When z_t ≈ 0: h_t ≈ h_{t-1}
  → Gradient flows through unchanged (like LSTM cell state)
• When z_t ≈ 1: h_t ≈ h̃_t
  → Gradient flows through tanh (like RNN)
• GRU learns when to preserve vs update!
```

### 3.2 GRU vs LSTM: Gradient Highway

```
┌────────────────────────────────────────────────────────────┐
│              LSTM vs GRU COMPARISON                        │
└────────────────────────────────────────────────────────────┘

LSTM:
  C_t = f_t ⊙ C_{t-1} + i_t ⊙ C̃_t
        ↑
  Separate cell state = gradient highway
  
  Pros:
  • Explicit memory (cell state)
  • More control (4 gates)
  • Better for very long sequences
  
  Cons:
  • More parameters
  • Slower training
  • More complex

GRU:
  h_t = (1 - z_t) ⊙ h_{t-1} + z_t ⊙ h̃_t
        ↑
  Hidden state itself = gradient highway
  
  Pros:
  • Fewer parameters (~25% less)
  • Faster training
  • Simpler architecture
  
  Cons:
  • Less control (2 gates)
  • May struggle with very long sequences

Performance:
  In practice, GRU and LSTM perform similarly on most tasks!
  Choice depends on:
  • Dataset size (GRU better for smaller datasets)
  • Sequence length (LSTM better for very long sequences)
  • Computational budget (GRU faster)
```

---

## 4. GRU Implementation

In [ ]:
def sigmoid(x):
    """Sigmoid activation: σ(x) = 1 / (1 + e^(-x))"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def tanh(x):
    """Hyperbolic tangent activation."""
    return np.tanh(x)

def softmax(x):
    """Softmax activation."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

print("✓ Activation functions defined")

In [ ]:
class GRU:
    """GRU implementation from scratch."""
    
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.001):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        
        # Initialize weights for reset gate
        self.W_r = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_r = np.zeros((hidden_dim, 1))
        
        # Initialize weights for update gate
        self.W_z = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_z = np.zeros((hidden_dim, 1))
        
        # Initialize weights for candidate hidden state
        self.W_h = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_h = np.zeros((hidden_dim, 1))
        
        # Initialize weights for output layer
        self.W_y = np.random.randn(output_dim, hidden_dim) * 0.01
        self.b_y = np.zeros((output_dim, 1))
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
    
    def forward(self, X):
        """Forward pass through GRU.
        
        Args:
            X: Input sequence (seq_length, input_dim)
        
        Returns:
            outputs: List of output probabilities
            cache: Dictionary containing intermediate values for backprop
        """
        seq_length = X.shape[0]
        
        # Initialize hidden state
        h = np.zeros((self.hidden_dim, 1))
        
        # Store values for backprop
        cache = {
            'h': [h],
            'r': [],
            'z': [],
            'h_tilde': [],
            'concat': [],
            'concat_reset': []
        }
        
        outputs = []
        
        for t in range(seq_length):
            # Get current input
            x_t = X[t].reshape(-1, 1)
            
            # Concatenate h and x
            concat = np.vstack([h, x_t])
            cache['concat'].append(concat)
            
            # Reset gate: decides how much past to use
            r_t = sigmoid(np.dot(self.W_r, concat) + self.b_r)
            cache['r'].append(r_t)
            
            # Update gate: decides how much to update
            z_t = sigmoid(np.dot(self.W_z, concat) + self.b_z)
            cache['z'].append(z_t)
            
            # Candidate hidden state: new information
            concat_reset = np.vstack([r_t * h, x_t])
            cache['concat_reset'].append(concat_reset)
            h_tilde_t = tanh(np.dot(self.W_h, concat_reset) + self.b_h)
            cache['h_tilde'].append(h_tilde_t)
            
            # Update hidden state: interpolate between old and new
            h = (1 - z_t) * h + z_t * h_tilde_t
            cache['h'].append(h)
            
            # Compute output
            logits = np.dot(self.W_y, h) + self.b_y
            y_t = softmax(logits.flatten())
            outputs.append(y_t)
        
        return outputs, cache
    
    def compute_loss(self, outputs, targets):
        """Compute cross-entropy loss."""
        seq_length = len(outputs)
        loss = 0.0
        for t in range(seq_length):
            loss += -np.log(outputs[t][targets[t]] + 1e-8)
        return loss / seq_length
    
    def backward(self, X, targets, outputs, cache):
        """Backpropagation through time for GRU."""
        seq_length = X.shape[0]
        
        # Initialize gradients
        dW_r = np.zeros_like(self.W_r)
        db_r = np.zeros_like(self.b_r)
        dW_z = np.zeros_like(self.W_z)
        db_z = np.zeros_like(self.b_z)
        dW_h = np.zeros_like(self.W_h)
        db_h = np.zeros_like(self.b_h)
        dW_y = np.zeros_like(self.W_y)
        db_y = np.zeros_like(self.b_y)
        
        dh_next = np.zeros((self.hidden_dim, 1))
        
        # Backprop through time
        for t in reversed(range(seq_length)):
            # Output layer gradient
            dy = outputs[t].copy()
            dy[targets[t]] -= 1
            dy = dy.reshape(-1, 1) / seq_length
            
            dW_y += np.dot(dy, cache['h'][t+1].T)
            db_y += dy
            
            # Hidden state gradient
            dh = np.dot(self.W_y.T, dy) + dh_next
            
            # Candidate hidden state gradient
            dh_tilde = dh * cache['z'][t]
            dh_tilde_raw = dh_tilde * (1 - cache['h_tilde'][t]**2)
            
            # Update gate gradient
            dz = dh * (cache['h_tilde'][t] - cache['h'][t])
            dz_raw = dz * cache['z'][t] * (1 - cache['z'][t])
            
            # Reset gate gradient
            dconcat_reset = np.dot(self.W_h.T, dh_tilde_raw)
            dr = dconcat_reset[:self.hidden_dim, :] * cache['h'][t]
            dr_raw = dr * cache['r'][t] * (1 - cache['r'][t])
            
            # Accumulate gradients
            dW_r += np.dot(dr_raw, cache['concat'][t].T)
            db_r += dr_raw
            dW_z += np.dot(dz_raw, cache['concat'][t].T)
            db_z += dz_raw
            dW_h += np.dot(dh_tilde_raw, cache['concat_reset'][t].T)
            db_h += dh_tilde_raw
            
            # Gradient for next iteration
            dconcat = (np.dot(self.W_r.T, dr_raw) + 
                      np.dot(self.W_z.T, dz_raw))
            dh_from_concat_reset = dconcat_reset[:self.hidden_dim, :] * cache['r'][t]
            dh_next = (dconcat[:self.hidden_dim, :] + 
                      dh_from_concat_reset + 
                      dh * (1 - cache['z'][t]))
        
        # Gradient clipping
        for grad in [dW_r, db_r, dW_z, db_z, dW_h, db_h, dW_y, db_y]:
            np.clip(grad, -5, 5, out=grad)
        
        return dW_r, db_r, dW_z, db_z, dW_h, db_h, dW_y, db_y
    
    def update_parameters(self, grads):
        """Update parameters using gradients."""
        dW_r, db_r, dW_z, db_z, dW_h, db_h, dW_y, db_y = grads
        
        self.W_r -= self.learning_rate * dW_r
        self.b_r -= self.learning_rate * db_r
        self.W_z -= self.learning_rate * dW_z
        self.b_z -= self.learning_rate * db_z
        self.W_h -= self.learning_rate * dW_h
        self.b_h -= self.learning_rate * db_h
        self.W_y -= self.learning_rate * dW_y
        self.b_y -= self.learning_rate * db_y
    
    def train_step(self, X, y):
        """Single training step."""
        outputs, cache = self.forward(X)
        loss = self.compute_loss(outputs, y)
        grads = self.backward(X, y, outputs, cache)
        self.update_parameters(grads)
        return loss, cache
    
    def predict(self, X):
        """Make predictions."""
        outputs, _ = self.forward(X)
        predictions = np.array([np.argmax(out) for out in outputs])
        return predictions
    
    def evaluate(self, X, y):
        """Evaluate on dataset."""
        total_loss = 0.0
        correct = 0
        total = 0
        
        for i in range(len(X)):
            outputs, _ = self.forward(X[i])
            loss = self.compute_loss(outputs, y[i])
            total_loss += loss
            
            predictions = np.array([np.argmax(out) for out in outputs])
            correct += np.sum(predictions == y[i])
            total += len(y[i])
        
        return total_loss / len(X), correct / total

print("✓ GRU class defined")
print("\n📊 GRU has 3 components (reset, update, candidate) + output layer")
print("   Approximately 25% fewer parameters than LSTM!")

---

## 5. Training and Comparison

Let's train GRU and compare with LSTM.

In [ ]:
# Create synthetic data for demonstration
n_samples = 100
seq_length = 50
input_dim = 13
n_classes = 10

X_train = np.random.randn(n_samples, seq_length, input_dim)
X_val = np.random.randn(20, seq_length, input_dim)
y_train = np.random.randint(0, n_classes, n_samples)
y_val = np.random.randint(0, n_classes, 20)

print("✓ Synthetic data created for demonstration")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")

In [ ]:
# Initialize GRU
hidden_dim = 64
output_dim = n_classes

gru = GRU(input_dim, hidden_dim, output_dim, learning_rate=0.001)

print("GRU initialized:")
print(f"  Input dim: {input_dim}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Output dim: {output_dim}")
print(f"\n  Parameters:")
print(f"    Reset gate:  {gru.W_r.size + gru.b_r.size}")
print(f"    Update gate: {gru.W_z.size + gru.b_z.size}")
print(f"    Candidate:   {gru.W_h.size + gru.b_h.size}")
print(f"    Output layer: {gru.W_y.size + gru.b_y.size}")
total_params = (gru.W_r.size + gru.b_r.size + gru.W_z.size + gru.b_z.size + 
                gru.W_h.size + gru.b_h.size + gru.W_y.size + gru.b_y.size)
print(f"    TOTAL: {total_params}")
print(f"\n  Note: ~25% fewer parameters than LSTM!")

In [ ]:
# Training loop
n_epochs = 20
print(f"Training GRU for {n_epochs} epochs...\n")

for epoch in range(n_epochs):
    # Shuffle training data
    indices = np.random.permutation(len(X_train))
    
    # Train
    for idx in indices:
        seq_label = y_train[idx]
        frame_labels = np.full(X_train[idx].shape[0], seq_label)
        gru.train_step(X_train[idx], frame_labels)
    
    # Evaluate
    train_loss, train_acc = gru.evaluate(
        X_train,
        np.array([np.full(X_train[i].shape[0], y_train[i]) for i in range(len(X_train))])
    )
    val_loss, val_acc = gru.evaluate(
        X_val,
        np.array([np.full(X_val[i].shape[0], y_val[i]) for i in range(len(X_val))])
    )
    
    # Store
    gru.train_losses.append(train_loss)
    gru.val_losses.append(val_loss)
    gru.train_accuracies.append(train_acc)
    gru.val_accuracies.append(val_acc)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("\n✓ Training complete!")

---

## 6. Summary and Conclusions

### What We Learned:

✅ **GRU simplifies LSTM** - Only 2 gates instead of 4  
✅ **Reset gate** - Controls how much past to use  
✅ **Update gate** - Controls how much to update  
✅ **Merged states** - No separate cell state  
✅ **Fewer parameters** - ~25% less than LSTM  
✅ **Similar performance** - Often matches LSTM  

### GRU vs LSTM: When to Use What?

```
┌────────────────────────────────────────────────────────────┐
│              DECISION GUIDE: GRU vs LSTM                   │
└────────────────────────────────────────────────────────────┘

Use GRU when:
  ✓ You have limited training data
  ✓ You need faster training
  ✓ Sequence length is moderate (<100 steps)
  ✓ You want simpler architecture
  ✓ Computational resources are limited

Use LSTM when:
  ✓ You have lots of training data
  ✓ Sequences are very long (>100 steps)
  ✓ You need maximum performance
  ✓ You have computational resources
  ✓ Task requires precise memory control

In practice:
  • Try both and see which works better!
  • GRU is often the better starting point
  • Performance difference is usually small
```

### Key Insights:

1. **Simplicity wins** - GRU's simpler design is often sufficient
2. **Fewer parameters** - Faster training, less overfitting
3. **Gradient highway** - Update gate creates path for gradients
4. **Practical choice** - GRU is often preferred in industry

### Historical Note:

- **LSTM** (1997): First gated RNN, solved vanishing gradient
- **GRU** (2014): Simplified LSTM, similar performance
- **Today**: Both widely used, choice depends on task

**GRU is the efficient alternative to LSTM!** 🚀